# imports

In [1]:
import numpy as np
import pandas as pd
import warnings

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

warnings.simplefilter('ignore')

# Config

In [2]:
BASE_PATH = "D:/projects/rahnamcollege-ml/demand-prediction/"

LABELED_DATA_PATH = BASE_PATH + 'data/label/label.parquet'
FEATURE_DATAFRAME_PATH = BASE_PATH + 'data/output/features.parquet'
START_DATE = '2023-01-09'
TEST_DATE = '2023-04-01'
LAST_DATE = '2023-05-01'
FEATURE_LIST = [
    'PU_day_of_week',
    'last_day_demand',
    'last_week_demand',
    'lag1-8',
    'lag2-9',
    'lag3-10',
    'lag4-11',
    'arima'
]
TARGET = 'label'
VALIDATION_SPLIT_RATIO = 0.2
LR_OUTPUT_PATH = BASE_PATH + 'data/output/lr_model_daily_result.parquet'
XGB_OUTPUT_PATH = BASE_PATH + 'data/output/xgboost_model_daily_results.parquet'

# Load Data

In [3]:
def load_labeled_data(path):
    return pd.read_parquet(path)


label_df = load_labeled_data(LABELED_DATA_PATH)
print(label_df.shape)
label_df.head()

(31964, 3)


,date,PULocationID,count
96,2023-01-02,1,32.0
358,2023-01-03,1,28.0
620,2023-01-04,1,8.0
882,2023-01-05,1,16.0
1144,2023-01-06,1,12.0


## adding calender features

In [4]:
def load_features(path):
    return pd.read_parquet(path)


feature_df = load_features(FEATURE_DATAFRAME_PATH)
print(feature_df.shape)
feature_df.head()

(28296, 13)


,date,PULocationID,arima,PU_day_of_month,week_of_month,PU_day_of_week,last_day_demand,last_week_demand,lag1-8,lag2-9,lag3-10,lag4-11,label
11,2023-01-13,1,0.916667,13,2,4,9.0,12.0,0.562500,1.250000,0.178571,0.468750,1.833333
12,2023-01-14,1,2.142857,14,2,5,22.0,7.0,1.833333,0.562500,1.250000,0.178571,1.142857
13,2023-01-15,1,0.846154,15,3,6,8.0,13.0,1.142857,1.833333,0.562500,1.250000,1.538462
14,2023-01-16,1,1.000000,16,3,0,20.0,15.0,1.538462,1.142857,1.833333,0.562500,1.466667
15,2023-01-17,1,3.000000,17,3,1,22.0,5.0,1.466667,1.538462,1.142857,1.833333,3.800000


### merge features and label

In [5]:
label_df['date'] = label_df['date'].astype(str)
feature_df['date'] = feature_df['date'].astype(str)

rides_df = pd.merge(label_df, feature_df, on=['date', 'PULocationID'])
rides_df.head()

,date,PULocationID,count,arima,PU_day_of_month,week_of_month,PU_day_of_week,last_day_demand,last_week_demand,lag1-8,lag2-9,lag3-10,lag4-11,label
0,2023-01-13,1,22.0,0.916667,13,2,4,9.0,12.0,0.562500,1.250000,0.178571,0.468750,1.833333
1,2023-01-14,1,8.0,2.142857,14,2,5,22.0,7.0,1.833333,0.562500,1.250000,0.178571,1.142857
2,2023-01-15,1,20.0,0.846154,15,3,6,8.0,13.0,1.142857,1.833333,0.562500,1.250000,1.538462
3,2023-01-16,1,22.0,1.000000,16,3,0,20.0,15.0,1.538462,1.142857,1.833333,0.562500,1.466667
4,2023-01-17,1,19.0,3.000000,17,3,1,22.0,5.0,1.466667,1.538462,1.142857,1.833333,3.800000


## checking one week of data as a sample

In [6]:
rides_df[(rides_df['PULocationID'] == 79)].head()

,date,PULocationID,count,arima,PU_day_of_month,week_of_month,PU_day_of_week,last_day_demand,last_week_demand,lag1-8,lag2-9,lag3-10,lag4-11,label
8424,2023-01-13,79,2771.0,1.117331,13,2,4,2024.0,2608.0,1.174014,1.158098,1.168135,1.067249,1.062500
8425,2023-01-14,79,5177.0,1.003131,14,2,5,2771.0,4471.0,1.062500,1.174014,1.158098,1.168135,1.157907
8426,2023-01-15,79,4366.0,1.114261,15,3,6,5177.0,3422.0,1.157907,1.062500,1.174014,1.158098,1.275862
8427,2023-01-16,79,1595.0,1.226678,16,3,0,4366.0,1222.0,1.275862,1.157907,1.062500,1.174014,1.305237
8428,2023-01-17,79,1408.0,0.963674,17,3,1,1595.0,1459.0,1.305237,1.275862,1.157907,1.062500,0.965045


## Dropping some samples

In [7]:
rides_df = rides_df.dropna()
rides_df = rides_df[rides_df['date'] < LAST_DATE]

print(rides_df.shape)
rides_df.head()

(28296, 14)


,date,PULocationID,count,arima,PU_day_of_month,week_of_month,PU_day_of_week,last_day_demand,last_week_demand,lag1-8,lag2-9,lag3-10,lag4-11,label
0,2023-01-13,1,22.0,0.916667,13,2,4,9.0,12.0,0.562500,1.250000,0.178571,0.468750,1.833333
1,2023-01-14,1,8.0,2.142857,14,2,5,22.0,7.0,1.833333,0.562500,1.250000,0.178571,1.142857
2,2023-01-15,1,20.0,0.846154,15,3,6,8.0,13.0,1.142857,1.833333,0.562500,1.250000,1.538462
3,2023-01-16,1,22.0,1.000000,16,3,0,20.0,15.0,1.538462,1.142857,1.833333,0.562500,1.466667
4,2023-01-17,1,19.0,3.000000,17,3,1,22.0,5.0,1.466667,1.538462,1.142857,1.833333,3.800000


## Train and Test split

In [8]:
def train_and_test_split(df: pd.DataFrame, split_date):
  train, test = df[df['date'] < split_date], df[df['date'] >= split_date]

  train.set_index('date', inplace = True)
  test.set_index('date', inplace = True)
  return train, test

train_df, test_df = train_and_test_split(rides_df, TEST_DATE)

print('train_df shape:', train_df.shape)
print('test_df shape:', test_df.shape)
train_df.head()

train_df shape: (20436, 13)
test_df shape: (7860, 13)


,PULocationID,count,arima,PU_day_of_month,week_of_month,PU_day_of_week,last_day_demand,last_week_demand,lag1-8,lag2-9,lag3-10,lag4-11,label
date,,,,,,,,,,,,,
2023-01-13,1,22.0,0.916667,13,2,4,9.0,12.0,0.562500,1.250000,0.178571,0.468750,1.833333
2023-01-14,1,8.0,2.142857,14,2,5,22.0,7.0,1.833333,0.562500,1.250000,0.178571,1.142857
2023-01-15,1,20.0,0.846154,15,3,6,8.0,13.0,1.142857,1.833333,0.562500,1.250000,1.538462
2023-01-16,1,22.0,1.000000,16,3,0,20.0,15.0,1.538462,1.142857,1.833333,0.562500,1.466667
2023-01-17,1,19.0,3.000000,17,3,1,22.0,5.0,1.466667,1.538462,1.142857,1.833333,3.800000


## Target and Feature split

In [9]:
train_set_label = train_df[TARGET]
train_set = train_df[FEATURE_LIST]

y_test = test_df[TARGET]
x_test = test_df[FEATURE_LIST]

## Train and Validation split

In [10]:
x_train, x_validation, y_train, y_validation = train_test_split(
    train_set, train_set_label, test_size=VALIDATION_SPLIT_RATIO, shuffle=True)

# ML Models

In [11]:
def model_training(ml_model, x_train, y_train, **params):
  model = ml_model(**params)
  model.fit(x_train, y_train)
  return model

replace_negatives = np.vectorize(lambda x : 1 if x < 1 else x)

## Calculate Error

In [12]:
def symmetric_mean_absolute_percentage_error(actual, predicted):
    res = np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual)) / 2))
    return round(res, 4)


def error_calculator(real_demand, predicted_demand):
  print('SMAPE: ', '{:.2%}'.format(symmetric_mean_absolute_percentage_error(real_demand, predicted_demand)))
  print('MAPE: ', '{:.2%}'.format(mean_absolute_percentage_error(real_demand, predicted_demand)))
  print('MSE: ', '{:.2f}'.format(mean_squared_error(real_demand, predicted_demand)))
  print('MAE: ', '{:.2f}'.format(mean_absolute_error(real_demand, predicted_demand)))


## Linear Regression Model

In [13]:
lr_model = model_training(LinearRegression, x_train, y_train)

### Validation prediction

In [14]:
lr_validation_pred = lr_model.predict(x_validation)
error_calculator(y_validation,lr_validation_pred)

SMAPE:  34.38%
MAPE:  44.60%
MSE:  0.37
MAE:  0.40


### Test prediction

In [15]:
lr_test_pred = lr_model.predict(x_test)
error_calculator(
    y_test * test_df['last_week_demand'], replace_negatives(lr_test_pred*test_df['last_week_demand']))

SMAPE:  32.72%
MAPE:  44.29%
MSE:  49570.61
MAE:  64.20


### Result Data

In [16]:
lr_result_df = test_df.copy()
lr_result_df.drop('count',axis=1,inplace=True)
lr_result_df['real demand'] = y_test * test_df['last_week_demand']
lr_result_df['predicted demand'] =replace_negatives( lr_test_pred * test_df['last_week_demand'])

print(lr_result_df.shape)
lr_result_df.head()

(7860, 14)


,PULocationID,arima,PU_day_of_month,week_of_month,PU_day_of_week,last_day_demand,last_week_demand,lag1-8,lag2-9,lag3-10,lag4-11,label,real demand,predicted demand
date,,,,,,,,,,,,,,
2023-04-01,1,0.812500,1,1,5,14.0,16.0,1.400000,0.636364,0.800000,2.142857,0.875000,14.0,18.631744
2023-04-02,1,0.866667,2,1,6,14.0,15.0,0.875000,1.400000,0.636364,0.800000,1.200000,18.0,17.689156
2023-04-03,1,3.500000,3,1,0,18.0,4.0,1.200000,0.875000,1.400000,0.636364,2.500000,10.0,10.527104
2023-04-04,1,0.800000,4,1,1,10.0,15.0,2.500000,1.200000,0.875000,1.400000,0.866667,13.0,18.596519
2023-04-05,1,1.625000,5,1,2,13.0,8.0,0.866667,2.500000,1.200000,0.875000,2.375000,19.0,13.364075


In [17]:
lr_result_df.to_parquet(LR_OUTPUT_PATH)

## XGBoost Model

### Hyperparameter tuning

In [18]:
def hyper_parameter_tuning(x_train, y_train, n_estimators, learning_rate, max_depth, scoring_method):
  parameters = {
      'n_estimators' : n_estimators,
      'learning_rate' : learning_rate,
      'max_depth' : max_depth
  }

  gc = GridSearchCV(XGBRegressor(), parameters, scoring=scoring_method)
  gc.fit(x_train, y_train)
  return gc.best_params_


n_estimators = [100,700, 1000]
learning_rate = [0.15, 0.1, 0.01]
max_depth = [3,5]
scoring_method = 'neg_root_mean_squared_error'

params = hyper_parameter_tuning(
    x_train,
    y_train,
    n_estimators,
    learning_rate,
    max_depth,
    scoring_method
)

print(params)

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}


### XGBoost Model

In [19]:
XGB_model = model_training(XGBRegressor, x_train, y_train, **params)

### Validation prediction

In [20]:
XGB_validation_pred = (XGB_model.predict(x_validation))
error_calculator(y_validation,XGB_validation_pred)

SMAPE:  27.02%
MAPE:  33.42%
MSE:  0.26
MAE:  0.32


### Test prediction

In [21]:
XGB_test_pred = (XGB_model.predict(x_test))
error_calculator(y_test * test_df['last_week_demand'],replace_negatives(XGB_test_pred * test_df['last_week_demand']))

SMAPE:  27.97%
MAPE:  34.50%
MSE:  12209.54
MAE:  36.30


### Result Data

In [22]:
XGB_result_df = test_df.copy()
XGB_result_df.drop('count',axis=1,inplace=True)
XGB_result_df['real demand'] = y_test * test_df['last_week_demand']
XGB_result_df['predicted demand'] = replace_negatives(XGB_test_pred * test_df['last_week_demand'])

print(XGB_result_df.shape)
XGB_result_df.head()

(7860, 14)


,PULocationID,arima,PU_day_of_month,week_of_month,PU_day_of_week,last_day_demand,last_week_demand,lag1-8,lag2-9,lag3-10,lag4-11,label,real demand,predicted demand
date,,,,,,,,,,,,,,
2023-04-01,1,0.812500,1,1,5,14.0,16.0,1.400000,0.636364,0.800000,2.142857,0.875000,14.0,15.044990
2023-04-02,1,0.866667,2,1,6,14.0,15.0,0.875000,1.400000,0.636364,0.800000,1.200000,18.0,14.524966
2023-04-03,1,3.500000,3,1,0,18.0,4.0,1.200000,0.875000,1.400000,0.636364,2.500000,10.0,11.454600
2023-04-04,1,0.800000,4,1,1,10.0,15.0,2.500000,1.200000,0.875000,1.400000,0.866667,13.0,14.461575
2023-04-05,1,1.625000,5,1,2,13.0,8.0,0.866667,2.500000,1.200000,0.875000,2.375000,19.0,12.628479


In [23]:
XGB_result_df.to_parquet(XGB_OUTPUT_PATH)